In [2]:
import polars as pl

In [3]:
df = pl.read_csv('data/uber_data.csv')

In [4]:
uber_data = df.with_columns(
    pl.col('tpep_pickup_datetime').str.to_datetime().alias('tpep_pickup_datetime'),
    pl.col('tpep_dropoff_datetime').str.to_datetime().alias('tpep_dropoff_datetime'),
).with_row_count('trip_id')

In [5]:
datetime_dim = uber_data.select(
    pl.col("tpep_pickup_datetime"),
    pl.col('tpep_pickup_datetime').dt.hour().alias('pickup_hour'),
    pl.col('tpep_pickup_datetime').dt.day().alias('pickup_day'),
    pl.col('tpep_pickup_datetime').dt.weekday().alias('pickup_weekday'),
    pl.col('tpep_pickup_datetime').dt.month().alias('pickup_month'),
    pl.col('tpep_pickup_datetime').dt.year().alias('pickup_year'),
    pl.col("tpep_dropoff_datetime"),
    pl.col('tpep_dropoff_datetime').dt.hour().alias('dropoff_hour'),
    pl.col('tpep_dropoff_datetime').dt.day().alias('dropoff_day'),
    pl.col('tpep_dropoff_datetime').dt.weekday().alias('dropoff_weekday'),
    pl.col('tpep_dropoff_datetime').dt.month().alias('dropoff_month'),
    pl.col('tpep_dropoff_datetime').dt.year().alias('dropoff_year')
).unique().with_row_count('datetime_id')


In [6]:
passenger_count_dim = uber_data.select(
    pl.col('passenger_count')
).unique().with_row_count('passenger_count_id')

# trip_distance_dim = uber_data.select(
#     pl.col('trip_distance')
# ).unique().with_row_count('trip_distance_id')

In [7]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}
rate_code_dim = uber_data.select(
    pl.col("RatecodeID"),
    pl.col("RatecodeID").map_dict(rate_code_type).alias("rate_code_name")
).unique().with_row_count('rate_code_id')

In [8]:
# pickup_loc_dim = uber_data.select(
#     pl.col('pickup_latitude'),
#     pl.col('pickup_longitude')
# ).unique().with_row_count('pickup_loc_id')

# dropoff_loc_dim = uber_data.select(
#     pl.col('dropoff_latitude'),
#     pl.col('dropoff_longitude')
# ).with_row_count('dropoff_loc_id')

In [9]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

payment_type_dim = uber_data.select(
    pl.col("payment_type"),
    pl.col("payment_type").map_dict(payment_type_name).alias("payment_type_name")
).unique().with_row_count('payment_type_id')

In [16]:
fact_table = uber_data.join(datetime_dim, on = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']) \
    .join(passenger_count_dim, on = 'passenger_count') \
    .join(rate_code_dim, on = 'RatecodeID')  \
    .join(payment_type_dim, on = 'payment_type') \
    .select(['trip_id','VendorID', 'datetime_id', 'passenger_count_id','payment_type_id', 
            'rate_code_id', 'store_and_fwd_flag', 
             # location can be stored in a separate dimension however for simplicity and improved transformation was kept as part of the fact
             'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude',
             'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
             'improvement_surcharge', 'total_amount'])

In [ ]:
fact_table[1:10,].to_dict()